In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Librerias

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input,Model
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from utils.metrics import *
from utils.preprocess import *
from utils.utils import get_weight

# Models

In [ ]:
def get_model_VGG16_gray():
    model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)

    # Block1_conv1 weights are of the format [3, 3, 3, 64] -> this is for RGB images
    # For grayscale, format should be [3, 3, 1, 64]. Weighted average of the features has to be calculated across channels.
    # RGB weights: Red 0.2989, Green 0.5870, Blue 0.1140

    # getting weights of block1 conv1.
    block1_conv1 = model.get_layer('block1_conv1').get_weights()
    weights, biases = block1_conv1

    # :weights shape = [3, 3, 3, 64] - (0, 1, 2, 3)
    # convert :weights shape to = [64, 3, 3, 3] - (3, 2, 0, 1)
    weights = np.transpose(weights, (3, 2, 0, 1))


    kernel_out_channels, kernel_in_channels, kernel_rows, kernel_columns = weights.shape

    # Dimensions : [kernel_out_channels, 1 (since grayscale), kernel_rows, kernel_columns]
    grayscale_weights = np.zeros((kernel_out_channels, 1, kernel_rows, kernel_columns))

      # iterate out_channels number of times
    for i in range(kernel_out_channels):

        # get kernel for every out_channel
        get_kernel = weights[i, :, :, :]

        temp_kernel = np.zeros((3, 3))

        # :get_kernel shape = [3, 3, 3]
        # axis, dims = (0, in_channel), (1, row), (2, col)

        # calculate weighted average across channel axis
        in_channels, in_rows, in_columns = get_kernel.shape

        for in_row in range(in_rows):
            for in_col in range(in_columns):
                feature_red = get_kernel[0, in_row, in_col]
                feature_green = get_kernel[1, in_row, in_col]
                feature_blue = get_kernel[2, in_row, in_col]

            # weighted average for RGB filter
            total = (feature_red * 0.2989) + (feature_green * 0.5870) + (feature_blue * 0.1140)

            temp_kernel[in_row, in_col] = total


        # :temp_kernel is a 3x3 matrix [rows x columns]
        # add an axis at the end to specify in_channel as 1

        # Second: Add axis at the start of :temp_kernel to make its shape: [1, 3, 3] which is [in_channel, rows, columns]
        temp_kernel = np.expand_dims(temp_kernel, axis=0)

        # Now, :temp_kernel shape is [1, 3, 3]

        # Concat :temp_kernel to :grayscale_weights along axis=0
        grayscale_weights[i, :, :, :] = temp_kernel

      # Dimension of :grayscale_weights is [64, 1, 3, 3]
      # In order to bring it to tensorflow or keras weight format, transpose :grayscale_weights

      # dimension, axis of :grayscale_weights = (out_channels: 0), (in_channels: 1), (rows: 2), (columns: 3)
      # tf format of weights = (rows: 0), (columns: 1), (in_channels: 2), (out_channels: 3)

      # Go from (0, 1, 2, 3) to (2, 3, 1, 0)
    grayscale_weights = np.transpose(grayscale_weights, (2, 3, 1, 0)) # (3, 3, 1, 64)

      # combine :grayscale_weights and :biases
    new_block1_conv1 = [grayscale_weights, biases]


      # Reconstruct the layers of VGG16 but replace block1_conv1 weights with :grayscale_weights

      # get weights of all the layers starting from 'block1_conv2'
    vgg16_weights = {}
    for layer in model.layers[2:]:
        if "conv" in layer.name:
            vgg16_weights["224_" + layer.name] = model.get_layer(layer.name).get_weights()

    del model


      # Custom build VGG16
    input = Input(shape=(224, 224, 1), name='224_input')
      # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 1), data_format="channels_last", name='224_block1_conv1')(input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='224_block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='224_block1_pool')(x)

      # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='224_block2_pool')(x)

      # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='224_block3_pool')(x)

      # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='224_block4_pool')(x)

      # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv3')(x)
    x = MaxPooling2D((8, 8), strides=(8, 8), name='224_block5_pool')(x)

    base_model = Model(inputs=input, outputs=x)

    base_model.get_layer('224_block1_conv1').set_weights(new_block1_conv1)
    for layer in base_model.layers[2:]:
        if 'conv' in layer.name:
            base_model.get_layer(layer.name).set_weights(vgg16_weights[layer.name])

    x = base_model.output

    for layer in base_model.layers:
        layer.trainable = True

    x = tf.keras.layers.GlobalAveragePooling2D()(x)  
    layers = tf.keras.layers.Flatten()(x)
      #layers = tf.keras.layers.Dense(128,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(1024 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense( 512,activation="relu")(layers)
    layers = tf.keras.layers.Dense( 64,activation="relu")(layers)
    predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layers)

      #Compilador
    model = tf.keras.Model(inputs = base_model.input, outputs=predictions)
    optimizer=tf.keras.optimizers.Adam(lr=0.0001) 
    loss='categorical_crossentropy'
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
def get_model_VGG19_gray():
      model = tf.keras.applications.VGG19(weights='imagenet', include_top=False)

      # Block1_conv1 weights are of the format [3, 3, 3, 64] -> this is for RGB images
      # For grayscale, format should be [3, 3, 1, 64]. Weighted average of the features has to be calculated across channels.
      # RGB weights: Red 0.2989, Green 0.5870, Blue 0.1140

      # getting weights of block1 conv1.
      block1_conv1 = model.get_layer('block1_conv1').get_weights()
      weights, biases = block1_conv1

      # :weights shape = [3, 3, 3, 64] - (0, 1, 2, 3)
      # convert :weights shape to = [64, 3, 3, 3] - (3, 2, 0, 1)
      weights = np.transpose(weights, (3, 2, 0, 1))


      kernel_out_channels, kernel_in_channels, kernel_rows, kernel_columns = weights.shape

      # Dimensions : [kernel_out_channels, 1 (since grayscale), kernel_rows, kernel_columns]
      grayscale_weights = np.zeros((kernel_out_channels, 1, kernel_rows, kernel_columns))

      # iterate out_channels number of times
      for i in range(kernel_out_channels):

        # get kernel for every out_channel
        get_kernel = weights[i, :, :, :]

        temp_kernel = np.zeros((3, 3))

        # :get_kernel shape = [3, 3, 3]
        # axis, dims = (0, in_channel), (1, row), (2, col)

        # calculate weighted average across channel axis
        in_channels, in_rows, in_columns = get_kernel.shape

        for in_row in range(in_rows):
          for in_col in range(in_columns):
            feature_red = get_kernel[0, in_row, in_col]
            feature_green = get_kernel[1, in_row, in_col]
            feature_blue = get_kernel[2, in_row, in_col]

            # weighted average for RGB filter
            total = (feature_red * 0.2989) + (feature_green * 0.5870) + (feature_blue * 0.1140)

            temp_kernel[in_row, in_col] = total


        # :temp_kernel is a 3x3 matrix [rows x columns]
        # add an axis at the end to specify in_channel as 1

        # Second: Add axis at the start of :temp_kernel to make its shape: [1, 3, 3] which is [in_channel, rows, columns]
        temp_kernel = np.expand_dims(temp_kernel, axis=0)

        # Now, :temp_kernel shape is [1, 3, 3]

        # Concat :temp_kernel to :grayscale_weights along axis=0
        grayscale_weights[i, :, :, :] = temp_kernel

      # Dimension of :grayscale_weights is [64, 1, 3, 3]
      # In order to bring it to tensorflow or keras weight format, transpose :grayscale_weights

      # dimension, axis of :grayscale_weights = (out_channels: 0), (in_channels: 1), (rows: 2), (columns: 3)
      # tf format of weights = (rows: 0), (columns: 1), (in_channels: 2), (out_channels: 3)

      # Go from (0, 1, 2, 3) to (2, 3, 1, 0)
      grayscale_weights = np.transpose(grayscale_weights, (2, 3, 1, 0)) # (3, 3, 1, 64)

      # combine :grayscale_weights and :biases
      new_block1_conv1 = [grayscale_weights, biases]


      # Reconstruct the layers of VGG16 but replace block1_conv1 weights with :grayscale_weights

      # get weights of all the layers starting from 'block1_conv2'
      vgg19_weights = {}
      for layer in model.layers[2:]:
        if "conv" in layer.name:
          vgg19_weights["224_" + layer.name] = model.get_layer(layer.name).get_weights()

      del model


      # Custom build VGG19
      input = Input(shape=(224, 224, 1), name='224_input')
      # Block 1
      x = Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 1), data_format="channels_last", name='224_block1_conv1')(input)
      x = Conv2D(64, (3, 3), activation='relu', padding='same', name='224_block1_conv2')(x)
      x = MaxPooling2D((2, 2), strides=(2, 2), name='224_block1_pool')(x)

      # Block 2
      x = Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block2_conv1')(x)
      x = Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block2_conv2')(x)
      x = MaxPooling2D((2, 2), strides=(2, 2), name='224_block2_pool')(x)

      # Block 3
      x = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv1')(x)
      x = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv2')(x)
      x = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv3')(x)
      x = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv4')(x)
      x = MaxPooling2D((2, 2), strides=(2, 2), name='224_block3_pool')(x)

      # Block 4
      x = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv1')(x)
      x = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv2')(x)
      x = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv3')(x)
      x = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv4')(x)
      x = MaxPooling2D((2, 2), strides=(2, 2), name='224_block4_pool')(x)

      # Block 5
      x = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv1')(x)
      x = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv2')(x)
      x = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv3')(x)
      x = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv4')(x)
      x = MaxPooling2D((8, 8), strides=(8, 8), name='224_block5_pool')(x)

      base_model = Model(inputs=input, outputs=x)

      base_model.get_layer('224_block1_conv1').set_weights(new_block1_conv1)
      for layer in base_model.layers[2:]:
        if 'conv' in layer.name:
          base_model.get_layer(layer.name).set_weights(vgg19_weights[layer.name])

      x = base_model.output

      for layer in base_model.layers:
          layer.trainable = True

      x = tf.keras.layers.GlobalAveragePooling2D()(x)  
      layers = tf.keras.layers.Flatten()(x)
      #layers = tf.keras.layers.Dense(128,activation="relu")(layers)
      layers = tf.keras.layers.Dropout(0.2)(layers)
      layers = tf.keras.layers.Dense(1024 ,activation="relu")(layers)
      layers = tf.keras.layers.Dropout(0.2)(layers)
      layers = tf.keras.layers.Dense( 512,activation="relu")(layers)
      layers = tf.keras.layers.Dense( 64,activation="relu")(layers)
      predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layers)

      #Compilador
      model = tf.keras.Model(inputs = base_model.input, outputs=predictions)
      optimizer=tf.keras.optimizers.Adam(lr=0.0001) 
      model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
      model.summary()
      return model

# BIMCV datasets

Source data can be downloaded from [Code Ocean Capsule](https://codeocean.com/capsule/8595156/tree).

In [ ]:
# Modify PATH accordingly.
Img_dir = 'data/DATABASES/CLASSIFICATION'

## Positive

In [ ]:
P_dir = Img_dir + '/POSITIVE'

## Negative Covid Era

In [ ]:
N1_dir = Img_dir + '/NEGATIVE COVID ERA'

## Negative Covid Era Without possible covid

In [ ]:
N2_dir = Img_dir + '/NEGATIVE COVID ERA WITHOUT POSSIBLE COVID IMAGES'

## Negative Pre-Covid Era

In [ ]:
N3_dir = Img_dir + '/NEGATIVE PRECOVID ERA'

# Experiment 1

### Part C

In [ ]:
C1_dir = '/Normal'

In [ ]:
X_train_P = np.load(P_dir + C1_dir + '/X_pos_train_C.npy')
X_test_P = np.load(P_dir + C1_dir + '/X_pos_test_C.npy')
X_dev_P = np.load(P_dir + C1_dir + '/X_pos_dev_C.npy')

In [ ]:
X_train_N = np.load(N1_dir + C1_dir + '/X_neg_train_C.npy')
X_test_N = np.load(N1_dir + C1_dir + '/X_neg_test_C.npy')
X_dev_N = np.load(N1_dir + C1_dir + '/X_neg_dev_C.npy')

In [ ]:
X_train=np.concatenate((X_train_N,X_train_P))
X_dev=np.concatenate((X_dev_N,X_dev_P))
X_test=np.concatenate((X_test_N,X_test_P))

In [ ]:
y_train_N=np.array([[1,0]]*len(X_train_N))
y_dev_N=np.array([[1,0]]*len(X_dev_N))
y_test_N=np.array([[1,0]]*len(X_test_N))

y_train_P=np.array([[0,1]]*len(X_train_P))
y_dev_P=np.array([[0,1]]*len(X_dev_P))
y_test_P=np.array([[0,1]]*len(X_test_P))

In [ ]:
y_train=np.concatenate((y_train_N,y_train_P))
y_dev=np.concatenate((y_dev_N,y_dev_P))
y_test=np.concatenate((y_test_N,y_test_P))

In [ ]:
#Min_max scaled
X_train,y_train=min_max_preprocessing(X_train,y_train)
X_dev,y_dev=min_max_preprocessing(X_dev,y_dev)
X_test,y_test=min_max_preprocessing(X_test,y_test)

In [ ]:
X_train = np.float32(X_train)
X_dev = np.float32(X_dev)
X_test = np.float32(X_test)

In [ ]:
#std normalization
X_train,y_train,mean,std=samplewise_preprocessing(X_train,y_train)

In [ ]:
X_dev=featurewise_preprocessing(X_dev,mean,std)
X_test=featurewise_preprocessing(X_test,mean,std)

#### Train Models (GPU) 

In [ ]:
log_dir="/content/drive/MyDrive/Covid19/COVID_FINAL/logs/"

In [ ]:
class_weight= get_weight(np.argmax(y_train,axis=-1))

In [ ]:
print("Getting Base Model")
model = get_model_VGG16_gray()

In [ ]:
model_name='VGG16_imagenet_Covid_pos_vs_neg'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

In [ ]:
print("Getting Base Model")
model = get_model_VGG19_gray()

In [ ]:
model_name='VGG19_imagenet_Covid_pos_vs_neg'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

#### Prediction

In [ ]:
print("Model VGG 16 Exp 1 Part C")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

In [ ]:
print("Model VGG 19 Exp 1 Part C")
model = get_model_VGG19_gray()
model.load_weights(log_dir+'VGG19_imagenet_Covid_pos_vs_neg')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

### Part D

In [ ]:
D1_dir = '/Segmented'

In [ ]:
X_train_P = np.load(P_dir + D1_dir + '/X_pos_Seg_Train.npy')
X_test_P = np.load(P_dir + D1_dir + '/X_pos_Seg_Test.npy')
X_dev_P = np.load(P_dir + D1_dir + '/X_pos_Seg_Dev.npy')

In [ ]:
X_train_N = np.load(N1_dir + D1_dir + '/X_neg_Seg_Train.npy')
X_test_N = np.load(N1_dir + D1_dir + '/X_neg_Seg_Test.npy')
X_dev_N = np.load(N1_dir + D1_dir + '/X_neg_Seg_Dev.npy')

In [ ]:
X_train=np.concatenate((X_train_N,X_train_P))
X_dev=np.concatenate((X_dev_N,X_dev_P))
X_test=np.concatenate((X_test_N,X_test_P))

In [ ]:
y_train_N=np.array([[1,0]]*len(X_train_N))
y_dev_N=np.array([[1,0]]*len(X_dev_N))
y_test_N=np.array([[1,0]]*len(X_test_N))

y_train_P=np.array([[0,1]]*len(X_train_P))
y_dev_P=np.array([[0,1]]*len(X_dev_P))
y_test_P=np.array([[0,1]]*len(X_test_P))

In [ ]:
y_train=np.concatenate((y_train_N,y_train_P))
y_dev=np.concatenate((y_dev_N,y_dev_P))
y_test=np.concatenate((y_test_N,y_test_P))

In [ ]:
#Min_max scaled
X_train,y_train=min_max_preprocessing(X_train,y_train)
X_dev,y_dev=min_max_preprocessing(X_dev,y_dev)
X_test,y_test=min_max_preprocessing(X_test,y_test)

In [ ]:
X_train = np.float32(X_train)
X_dev = np.float32(X_dev)
X_test = np.float32(X_test)

In [ ]:
#std normalization
X_train,y_train,mean,std=samplewise_preprocessing(X_train,y_train)

In [ ]:
X_dev=featurewise_preprocessing(X_dev,mean,std)
X_test=featurewise_preprocessing(X_test,mean,std)

#### Train Models (GPU) 

In [ ]:
log_dir="/content/drive/MyDrive/Covid19/COVID_FINAL/logs/"

In [ ]:
class_weight= get_weight(np.argmax(y_train,axis=-1))

In [ ]:
print("Getting Base Model")
model = get_model_VGG16_gray()

In [ ]:
model_name='VGG16_imagenet_Covid_pos_vs_neg_seg'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

In [ ]:
print("Getting Base Model")
model = get_model_VGG19_gray()

In [ ]:
model_name='VGG19_imagenet_Covid_pos_vs_neg_seg'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

#### Prediction

In [ ]:
print("Model VGG 16 Exp 1 Part D")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg_seg')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

In [ ]:
print("Model VGG 19 Exp 1 Part D")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg_seg')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

### Part E

In [ ]:
E1_dir = '/Inverted'

In [ ]:
X_train_P = np.load(P_dir + E1_dir + '/X_pos_Seg_i2_Train.npy')
X_test_P = np.load(P_dir + E1_dir + '/X_pos_Seg_i2_Test.npy')
X_dev_P = np.load(P_dir + E1_dir + '/X_pos_Seg_i2_Dev.npy')

In [ ]:
X_train_N = np.load(N1_dir + E1_dir + '/X_neg_Seg_i_Train.npy')
X_test_N = np.load(N1_dir + E1_dir + '/X_neg_Seg_i_Test.npy')
X_dev_N = np.load(N1_dir + E1_dir + '/X_neg_Seg_i_Dev.npy')

In [ ]:
X_train=np.concatenate((X_train_N,X_train_P))
X_dev=np.concatenate((X_dev_N,X_dev_P))
X_test=np.concatenate((X_test_N,X_test_P))

In [ ]:
y_train_N=np.array([[1,0]]*len(X_train_N))
y_dev_N=np.array([[1,0]]*len(X_dev_N))
y_test_N=np.array([[1,0]]*len(X_test_N))

y_train_P=np.array([[0,1]]*len(X_train_P))
y_dev_P=np.array([[0,1]]*len(X_dev_P))
y_test_P=np.array([[0,1]]*len(X_test_P))

In [ ]:
y_train=np.concatenate((y_train_N,y_train_P))
y_dev=np.concatenate((y_dev_N,y_dev_P))
y_test=np.concatenate((y_test_N,y_test_P))

In [ ]:
#Min_max scaled
X_train,y_train=min_max_preprocessing(X_train,y_train)
X_dev,y_dev=min_max_preprocessing(X_dev,y_dev)
X_test,y_test=min_max_preprocessing(X_test,y_test)

In [ ]:
X_train = np.float32(X_train)
X_dev = np.float32(X_dev)
X_test = np.float32(X_test)

In [ ]:
#std normalization
X_train,y_train,mean,std=samplewise_preprocessing(X_train,y_train)

In [ ]:
X_dev=featurewise_preprocessing(X_dev,mean,std)
X_test=featurewise_preprocessing(X_test,mean,std)

#### Train Models (GPU) 

In [ ]:
log_dir="/content/drive/MyDrive/Covid19/COVID_FINAL/logs/"

In [ ]:
class_weight= get_weight(np.argmax(y_train,axis=-1))

In [ ]:
print("Getting Base Model")
model = get_model_VGG16_gray()

In [ ]:
model_name='VGG16_imagenet_Covid_pos_vs_neg_inv'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

In [ ]:
print("Getting Base Model")
model = get_model_VGG19_gray()

In [ ]:
model_name='VGG19_imagenet_Covid_pos_vs_neg_inv'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

#### Prediction

In [ ]:
print("Model VGG 16 Exp 1 Part E")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg_inv')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

In [ ]:
print("Model VGG 19 Exp 1 Part E")
model = get_model_VGG19_gray()
model.load_weights(log_dir+'VGG19_imagenet_Covid_pos_vs_neg_inv')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

## Experiment 2

### Part C

In [ ]:
C1_dir = '/Normal'

In [ ]:
X_train_P = np.load(P_dir + C1_dir + '/X_pos_train_C.npy')
X_test_P = np.load(P_dir + C1_dir + '/X_pos_test_C.npy')
X_dev_P = np.load(P_dir + C1_dir + '/X_pos_dev_C.npy')

In [ ]:
X_train_N = np.load(N2_dir + C1_dir + '/X_neg_Train_CC.npy')
X_test_N = np.load(N2_dir + C1_dir + '/X_neg_Test_CC.npy')
X_dev_N = np.load(N2_dir + C1_dir + '/X_neg_Dev_CC.npy')

In [ ]:
X_train=np.concatenate((X_train_N,X_train_P))
X_dev=np.concatenate((X_dev_N,X_dev_P))
X_test=np.concatenate((X_test_N,X_test_P))

In [ ]:
y_train_N=np.array([[1,0]]*len(X_train_N))
y_dev_N=np.array([[1,0]]*len(X_dev_N))
y_test_N=np.array([[1,0]]*len(X_test_N))

y_train_P=np.array([[0,1]]*len(X_train_P))
y_dev_P=np.array([[0,1]]*len(X_dev_P))
y_test_P=np.array([[0,1]]*len(X_test_P))

In [ ]:
y_train=np.concatenate((y_train_N,y_train_P))
y_dev=np.concatenate((y_dev_N,y_dev_P))
y_test=np.concatenate((y_test_N,y_test_P))

In [ ]:
#Min_max scaled
X_train,y_train=min_max_preprocessing(X_train,y_train)
X_dev,y_dev=min_max_preprocessing(X_dev,y_dev)
X_test,y_test=min_max_preprocessing(X_test,y_test)

In [ ]:
X_train = np.float32(X_train)
X_dev = np.float32(X_dev)
X_test = np.float32(X_test)

In [ ]:
#std normalization
X_train,y_train,mean,std=samplewise_preprocessing(X_train,y_train)

In [ ]:
X_dev=featurewise_preprocessing(X_dev,mean,std)
X_test=featurewise_preprocessing(X_test,mean,std)

#### Train Models (GPU) 

In [ ]:
log_dir="/content/drive/MyDrive/Covid19/COVID_FINAL/logs/"

In [ ]:
class_weight= get_weight(np.argmax(y_train,axis=-1))

In [ ]:
print("Getting Base Model")
model = get_model_VGG16_gray()

In [ ]:
model_name='VGG16_imagenet_Covid_pos_vs_neg2'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

In [ ]:
print("Getting Base Model")
model = get_model_VGG19_gray()

In [ ]:
model_name='VGG19_imagenet_Covid_pos_vs_neg2'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

#### Prediction

In [ ]:
print("Model VGG 16 Exp 2 Part C")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg2')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

In [ ]:
print("Model VGG 19 Exp 2 Part C")
model = get_model_VGG19_gray()
model.load_weights(log_dir+'VGG19_imagenet_Covid_pos_vs_neg2')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

### Part D

In [ ]:
D1_dir = '/Segmented'

In [ ]:
X_train_P = np.load(P_dir + D1_dir + '/X_pos_Seg_Train.npy')
X_test_P = np.load(P_dir + D1_dir + '/X_pos_Seg_Test.npy')
X_dev_P = np.load(P_dir + D1_dir + '/X_pos_Seg_Dev.npy')

In [ ]:
X_train_N = np.load(N2_dir + D1_dir + '/X_neg_Seg_Train_2.npy')
X_test_N = np.load(N2_dir + D1_dir + '/X_neg_Seg_Test_2.npy')
X_dev_N = np.load(N2_dir + D1_dir + '/X_neg_Seg_Dev_2.npy')

In [ ]:
X_train=np.concatenate((X_train_N,X_train_P))
X_dev=np.concatenate((X_dev_N,X_dev_P))
X_test=np.concatenate((X_test_N,X_test_P))

In [ ]:
y_train_N=np.array([[1,0]]*len(X_train_N))
y_dev_N=np.array([[1,0]]*len(X_dev_N))
y_test_N=np.array([[1,0]]*len(X_test_N))

y_train_P=np.array([[0,1]]*len(X_train_P))
y_dev_P=np.array([[0,1]]*len(X_dev_P))
y_test_P=np.array([[0,1]]*len(X_test_P))

In [ ]:
y_train=np.concatenate((y_train_N,y_train_P))
y_dev=np.concatenate((y_dev_N,y_dev_P))
y_test=np.concatenate((y_test_N,y_test_P))

In [ ]:
#Min_max scaled
X_train,y_train=min_max_preprocessing(X_train,y_train)
X_dev,y_dev=min_max_preprocessing(X_dev,y_dev)
X_test,y_test=min_max_preprocessing(X_test,y_test)

In [ ]:
X_train = np.float32(X_train)
X_dev = np.float32(X_dev)
X_test = np.float32(X_test)

In [ ]:
#std normalization
X_train,y_train,mean,std=samplewise_preprocessing(X_train,y_train)

In [ ]:
X_dev=featurewise_preprocessing(X_dev,mean,std)
X_test=featurewise_preprocessing(X_test,mean,std)

#### Train Models (GPU) 

In [ ]:
log_dir="/content/drive/MyDrive/Covid19/COVID_FINAL/logs/"

In [ ]:
class_weight= get_weight(np.argmax(y_train,axis=-1))

In [ ]:
print("Getting Base Model")
model = get_model_VGG16_gray()

In [ ]:
model_name='VGG16_imagenet_posyneg_seg_C'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[checkpoint])

In [ ]:
print("Getting Base Model")
model = get_model_VGG19_gray()

In [ ]:
model_name='VGG19_imagenet_Covid_pos_vs_neg2_seg'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

#### Prediction

In [ ]:
print("Model VGG 16 Exp 2 Part D")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg2_seg')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

In [ ]:
print("Model VGG 19 Exp 2 Part D")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg2_seg')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

### Part E

In [ ]:
E1_dir = '/Inverted'

In [ ]:
X_train_P = np.load(P_dir + E1_dir + '/X_pos_Seg_i2_Train.npy')
X_test_P = np.load(P_dir + E1_dir + '/X_pos_Seg_i2_Test.npy')
X_dev_P = np.load(P_dir + E1_dir + '/X_pos_Seg_i2_Dev.npy')

In [ ]:
X_train_N = np.load(N2_dir + E1_dir + '/X_neg_Seg_Train_i2.npy')
X_test_N = np.load(N2_dir + E1_dir + '/X_neg_Seg_Test_i2.npy')
X_dev_N = np.load(N2_dir + E1_dir + '/X_neg_Seg_Dev_i2.npy')

In [ ]:
X_train=np.concatenate((X_train_N,X_train_P))
X_dev=np.concatenate((X_dev_N,X_dev_P))
X_test=np.concatenate((X_test_N,X_test_P))

In [ ]:
y_train_N=np.array([[1,0]]*len(X_train_N))
y_dev_N=np.array([[1,0]]*len(X_dev_N))
y_test_N=np.array([[1,0]]*len(X_test_N))

y_train_P=np.array([[0,1]]*len(X_train_P))
y_dev_P=np.array([[0,1]]*len(X_dev_P))
y_test_P=np.array([[0,1]]*len(X_test_P))

In [ ]:
y_train=np.concatenate((y_train_N,y_train_P))
y_dev=np.concatenate((y_dev_N,y_dev_P))
y_test=np.concatenate((y_test_N,y_test_P))

In [ ]:
#Min_max scaled
X_train,y_train=min_max_preprocessing(X_train,y_train)
X_dev,y_dev=min_max_preprocessing(X_dev,y_dev)
X_test,y_test=min_max_preprocessing(X_test,y_test)

In [ ]:
X_train = np.float32(X_train)
X_dev = np.float32(X_dev)
X_test = np.float32(X_test)

In [ ]:
#std normalization
X_train,y_train,mean,std=samplewise_preprocessing(X_train,y_train)

In [ ]:
X_dev=featurewise_preprocessing(X_dev,mean,std)
X_test=featurewise_preprocessing(X_test,mean,std)

#### Train Models (GPU) 

In [ ]:
log_dir="/content/drive/MyDrive/Covid19/COVID_FINAL/logs/"

In [ ]:
class_weight= get_weight(np.argmax(y_train,axis=-1))

In [ ]:
print("Getting Base Model")
model = get_model_VGG16_gray()

In [ ]:
model_name='VGG16_imagenet_Covid_pos_vs_neg2_inv'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

In [ ]:
print("Getting Base Model")
model = get_model_VGG19_gray()

In [ ]:
model_name='VGG19_imagenet_Covid_pos_vs_neg2_inv'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

#### Prediction

In [ ]:
print("Model VGG 16 Exp 2 Part E")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg2_inv')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

In [ ]:
print("Model VGG 19 Exp 2 Part E")
model = get_model_VGG19_gray()
model.load_weights(log_dir+'VGG19_imagenet_Covid_pos_vs_neg2_inv')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

## Experiment 3

### Part C

In [ ]:
C1_dir = '/Normal'

In [ ]:
X_train_P = np.load(P_dir + C1_dir + '/X_pos_train_C.npy')
X_test_P = np.load(P_dir + C1_dir + '/X_pos_test_C.npy')
X_dev_P = np.load(P_dir + C1_dir + '/X_pos_dev_C.npy')

In [ ]:
X_train_N = np.load(N3_dir + C1_dir + '/X_train_COVID_neg2.npy')
X_test_N = np.load(N3_dir + C1_dir + '/X_test_COVID_neg2.npy')
X_dev_N = np.load(N3_dir + C1_dir + '/X_dev_COVID_neg2.npy')

In [ ]:
X_train=np.concatenate((X_train_N,X_train_P))
X_dev=np.concatenate((X_dev_N,X_dev_P))
X_test=np.concatenate((X_test_N,X_test_P))

In [ ]:
y_train_N=np.array([[1,0]]*len(X_train_N))
y_dev_N=np.array([[1,0]]*len(X_dev_N))
y_test_N=np.array([[1,0]]*len(X_test_N))

y_train_P=np.array([[0,1]]*len(X_train_P))
y_dev_P=np.array([[0,1]]*len(X_dev_P))
y_test_P=np.array([[0,1]]*len(X_test_P))

In [ ]:
y_train=np.concatenate((y_train_N,y_train_P))
y_dev=np.concatenate((y_dev_N,y_dev_P))
y_test=np.concatenate((y_test_N,y_test_P))

In [ ]:
#Min_max scaled
X_train,y_train=min_max_preprocessing(X_train,y_train)
X_dev,y_dev=min_max_preprocessing(X_dev,y_dev)
X_test,y_test=min_max_preprocessing(X_test,y_test)

In [ ]:
X_train = np.float32(X_train)
X_dev = np.float32(X_dev)
X_test = np.float32(X_test)

In [ ]:
#std normalization
X_train,y_train,mean,std=samplewise_preprocessing(X_train,y_train)

In [ ]:
X_dev=featurewise_preprocessing(X_dev,mean,std)
X_test=featurewise_preprocessing(X_test,mean,std)

#### Train Models (GPU) 

In [ ]:
log_dir="/content/drive/MyDrive/Covid19/COVID_FINAL/logs/"

In [ ]:
class_weight= get_weight(np.argmax(y_train,axis=-1))

In [ ]:
print("Getting Base Model")
model = get_model_VGG16_gray()

In [ ]:
model_name='VGG16_imagenet_Covid_pos_vs_neg3'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

In [ ]:
print("Getting Base Model")
model = get_model_VGG19_gray()

In [ ]:
model_name='VGG19_imagenet_Covid_pos_vs_neg3'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

#### Prediction

In [ ]:
print("Model VGG 16 Exp 3 Part C")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg3')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

In [ ]:
print("Model VGG 19 Exp 3 Part C")
model = get_model_VGG19_gray()
model.load_weights(log_dir+'VGG19_imagenet_Covid_pos_vs_neg3')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

### Part D

In [ ]:
D1_dir = '/Segmented'

In [ ]:
X_train_P = np.load(P_dir + D1_dir + '/X_pos_Seg_Train.npy')
X_test_P = np.load(P_dir + D1_dir + '/X_pos_Seg_Test.npy')
X_dev_P = np.load(P_dir + D1_dir + '/X_pos_Seg_Dev.npy')

In [ ]:
X_train_N = np.load(N3_dir + D1_dir + '/X_preC_Seg_Train.npy')
X_test_N = np.load(N3_dir + D1_dir + '/X_preC_Seg_Test.npy')
X_dev_N = np.load(N3_dir + D1_dir + '/X_preC_Seg_Dev.npy')

In [ ]:
X_train=np.concatenate((X_train_N,X_train_P))
X_dev=np.concatenate((X_dev_N,X_dev_P))
X_test=np.concatenate((X_test_N,X_test_P))

In [ ]:
y_train_N=np.array([[1,0]]*len(X_train_N))
y_dev_N=np.array([[1,0]]*len(X_dev_N))
y_test_N=np.array([[1,0]]*len(X_test_N))

y_train_P=np.array([[0,1]]*len(X_train_P))
y_dev_P=np.array([[0,1]]*len(X_dev_P))
y_test_P=np.array([[0,1]]*len(X_test_P))

In [ ]:
y_train=np.concatenate((y_train_N,y_train_P))
y_dev=np.concatenate((y_dev_N,y_dev_P))
y_test=np.concatenate((y_test_N,y_test_P))

In [ ]:
#Min_max scaled
X_train,y_train=min_max_preprocessing(X_train,y_train)
X_dev,y_dev=min_max_preprocessing(X_dev,y_dev)
X_test,y_test=min_max_preprocessing(X_test,y_test)

In [ ]:
X_train = np.float32(X_train)
X_dev = np.float32(X_dev)
X_test = np.float32(X_test)

In [ ]:
#std normalization
X_train,y_train,mean,std=samplewise_preprocessing(X_train,y_train)

In [ ]:
X_dev=featurewise_preprocessing(X_dev,mean,std)
X_test=featurewise_preprocessing(X_test,mean,std)

#### Train Models (GPU) 

In [ ]:
log_dir="/content/drive/MyDrive/Covid19/COVID_FINAL/logs/"

In [ ]:
class_weight= get_weight(np.argmax(y_train,axis=-1))

In [ ]:
print("Getting Base Model")
model = get_model_VGG16_gray()

In [ ]:
model_name='VGG16_imagenet_Covid_pos_vs_neg3_seg'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

In [ ]:
print("Getting Base Model")
model = get_model_VGG19_gray()

In [ ]:
model_name='VGG19_imagenet_Covid_pos_vs_neg3_seg'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

#### Prediction

In [ ]:
print("Model VGG 16 Exp 3 Part D")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg3_seg')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

In [ ]:
print("Model VGG 19 Exp 3 Part D")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg3_seg')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

### Part E

In [ ]:
E1_dir = '/Inverted'

In [ ]:
X_train_P = np.load(P_dir + E1_dir + '/X_pos_Seg_i2_Train.npy')
X_test_P = np.load(P_dir + E1_dir + '/X_pos_Seg_i2_Test.npy')
X_dev_P = np.load(P_dir + E1_dir + '/X_pos_Seg_i2_Dev.npy')

In [ ]:
X_train_N = np.load(N3_dir + E1_dir + '/X_preC_Seg_Train_i.npy')
X_test_N = np.load(N3_dir + E1_dir + '/X_preC_Seg_Test_i.npy')
X_dev_N = np.load(N3_dir + E1_dir + '/X_preC_Seg_Dev_i.npy')

In [ ]:
X_train=np.concatenate((X_train_N,X_train_P))
X_dev=np.concatenate((X_dev_N,X_dev_P))
X_test=np.concatenate((X_test_N,X_test_P))

In [ ]:
y_train_N=np.array([[1,0]]*len(X_train_N))
y_dev_N=np.array([[1,0]]*len(X_dev_N))
y_test_N=np.array([[1,0]]*len(X_test_N))

y_train_P=np.array([[0,1]]*len(X_train_P))
y_dev_P=np.array([[0,1]]*len(X_dev_P))
y_test_P=np.array([[0,1]]*len(X_test_P))

In [ ]:
y_train=np.concatenate((y_train_N,y_train_P))
y_dev=np.concatenate((y_dev_N,y_dev_P))
y_test=np.concatenate((y_test_N,y_test_P))

In [ ]:
#Min_max scaled
X_train,y_train=min_max_preprocessing(X_train,y_train)
X_dev,y_dev=min_max_preprocessing(X_dev,y_dev)
X_test,y_test=min_max_preprocessing(X_test,y_test)

In [ ]:
X_train = np.float32(X_train)
X_dev = np.float32(X_dev)
X_test = np.float32(X_test)

In [ ]:
#std normalization
X_train,y_train,mean,std=samplewise_preprocessing(X_train,y_train)

In [ ]:
X_dev=featurewise_preprocessing(X_dev,mean,std)
X_test=featurewise_preprocessing(X_test,mean,std)

#### Train Models (GPU) 

In [ ]:
log_dir="/content/drive/MyDrive/Covid19/COVID_FINAL/logs/"

In [ ]:
class_weight= get_weight(np.argmax(y_train,axis=-1))

In [ ]:
print("Getting Base Model")
model = get_model_VGG16_gray()

In [ ]:
model_name='VGG16_imagenet_Covid_pos_vs_neg3_inv'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

In [ ]:
print("Getting Base Model")
model = get_model_VGG19_gray()

In [ ]:
model_name='VGG19_imagenet_Covid_pos_vs_neg3_inv'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

#### Prediction

In [ ]:
print("Model VGG 16 Exp 3 Part E")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg3_inv')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

In [ ]:
print("Model VGG 19 Exp 3 Part E")
model = get_model_VGG19_gray()
model.load_weights(log_dir+'VGG19_imagenet_Covid_pos_vs_neg3_inv')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

# Comparison Datasets

Cohen(Positive)

In [ ]:
P_dir = Img_dir + '/POSITIVE COHEN'

Negative(Normal/Pneumonia)

In [ ]:
N_dir = Img_dir + '/NEGATIVE ONLY PNEUMO-NORMAL FOR PAPER'

### Part C

In [ ]:
X_P = np.load(P_dir + '/X_Covid_Cohen_C.npy')
X_train_P,X_test_P,X_dev_P = np.array_split(X_P,[int(0.7*len(X_P)),int(0.85*len(X_P))])

In [ ]:
X_N1 = np.load(N_dir+'/X_Covid_Normal.npy')
X_train_N1,X_test_N1,X_dev_N1 = np.array_split(X_N1,[int(0.7*len(X_N1)),int(0.85*len(X_N1))])
X_N2 = np.load(N_dir'/X_Covid_Pneumo.npy')
X_train_N2,X_test_N2,X_dev_N2 = np.array_split(X_N2,[int(0.7*len(X_N2)),int(0.85*len(X_N2))])

In [ ]:
X_train_N=np.concatenate((X_train_N1,X_train_N2))
X_dev_N=np.concatenate((X_dev_N1,X_dev_N2))
X_test_N=np.concatenate((X_test_N1,X_test_N2))

In [ ]:
X_train=np.concatenate((X_train_N,X_train_P))
X_dev=np.concatenate((X_dev_N,X_dev_P))
X_test=np.concatenate((X_test_N,X_test_P))

In [ ]:
y_train_N=np.array([[1,0]]*len(X_train_N))
y_dev_N=np.array([[1,0]]*len(X_dev_N))
y_test_N=np.array([[1,0]]*len(X_test_N))

y_train_P=np.array([[0,1]]*len(X_train_P))
y_dev_P=np.array([[0,1]]*len(X_dev_P))
y_test_P=np.array([[0,1]]*len(X_test_P))

In [ ]:
y_train=np.concatenate((y_train_N,y_train_P))
y_dev=np.concatenate((y_dev_N,y_dev_P))
y_test=np.concatenate((y_test_N,y_test_P))

In [ ]:
#Min_max scaled
X_train,y_train=min_max_preprocessing(X_train,y_train)
X_dev,y_dev=min_max_preprocessing(X_dev,y_dev)
X_test,y_test=min_max_preprocessing(X_test,y_test)

In [ ]:
X_train = np.float32(X_train)
X_dev = np.float32(X_dev)
X_test = np.float32(X_test)

In [ ]:
#std normalization
X_train,y_train,mean,std=samplewise_preprocessing(X_train,y_train)

In [ ]:
X_dev=featurewise_preprocessing(X_dev,mean,std)
X_test=featurewise_preprocessing(X_test,mean,std)

#### Train Models (GPU) 

In [ ]:
log_dir="/content/drive/MyDrive/Covid19/COVID_FINAL/logs/"

In [ ]:
class_weight= get_weight(np.argmax(y_train,axis=-1))

In [ ]:
print("Getting Base Model")
model = get_model_VGG16_gray()

In [ ]:
model_name='VGG16_imagenet_Covid_pos_vs_neg_Comp'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

In [ ]:
print("Getting Base Model")
model = get_model_VGG19_gray()

In [ ]:
model_name='VGG19_imagenet_Covid_pos_vs_neg_Comp'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

#### Prediction

In [ ]:
print("Model VGG 16 Comp Part C")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg_Comp')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

In [ ]:
print("Model VGG 19 Comp Part C")
model = get_model_VGG19_gray()
model.load_weights(log_dir+'VGG19_imagenet_Covid_pos_vs_neg_Comp')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

### Part D

In [ ]:
X_P = np.load(P_dir + '/X_Covid_Cohen_C_Seg.npy')
X_train_P,X_test_P,X_dev_P = np.array_split(X_P,[int(0.7*len(X_P)),int(0.85*len(X_P))])

In [ ]:
X_N1 = np.load(N_dir+'/X_Covid_Normal.npy')
X_train_N1,X_test_N1,X_dev_N1 = np.array_split(X_N1,[int(0.7*len(X_N1)),int(0.85*len(X_N1))])
X_N2 = np.load(N_dir'/X_Covid_Pneumo.npy')
X_train_N2,X_test_N2,X_dev_N2 = np.array_split(X_N2,[int(0.7*len(X_N2)),int(0.85*len(X_N2))])

In [ ]:
X_train_N=np.concatenate((X_train_N1,X_train_N2))
X_dev_N=np.concatenate((X_dev_N1,X_dev_N2))
X_test_N=np.concatenate((X_test_N1,X_test_N2))

In [ ]:
X_train=np.concatenate((X_train_N,X_train_P))
X_dev=np.concatenate((X_dev_N,X_dev_P))
X_test=np.concatenate((X_test_N,X_test_P))

In [ ]:
y_train_N=np.array([[1,0]]*len(X_train_N))
y_dev_N=np.array([[1,0]]*len(X_dev_N))
y_test_N=np.array([[1,0]]*len(X_test_N))

y_train_P=np.array([[0,1]]*len(X_train_P))
y_dev_P=np.array([[0,1]]*len(X_dev_P))
y_test_P=np.array([[0,1]]*len(X_test_P))

In [ ]:
y_train=np.concatenate((y_train_N,y_train_P))
y_dev=np.concatenate((y_dev_N,y_dev_P))
y_test=np.concatenate((y_test_N,y_test_P))

In [ ]:
#Min_max scaled
X_train,y_train=min_max_preprocessing(X_train,y_train)
X_dev,y_dev=min_max_preprocessing(X_dev,y_dev)
X_test,y_test=min_max_preprocessing(X_test,y_test)

In [ ]:
X_train = np.float32(X_train)
X_dev = np.float32(X_dev)
X_test = np.float32(X_test)

In [ ]:
#std normalization
X_train,y_train,mean,std=samplewise_preprocessing(X_train,y_train)

In [ ]:
X_dev=featurewise_preprocessing(X_dev,mean,std)
X_test=featurewise_preprocessing(X_test,mean,std)

#### Train Models (GPU) 

In [ ]:
log_dir="/content/drive/MyDrive/Covid19/COVID_FINAL/logs/"

In [ ]:
class_weight= get_weight(np.argmax(y_train,axis=-1))

In [ ]:
print("Getting Base Model")
model = get_model_VGG16_gray()

In [ ]:
model_name='VGG16_imagenet_Covid_pos_vs_neg_seg_Comp'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

In [ ]:
print("Getting Base Model")
model = get_model_VGG19_gray()

In [ ]:
model_name='VGG19_imagenet_Covid_pos_vs_neg_seg_Comp'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

#### Prediction

In [ ]:
print("Model VGG 16 Comp Part D")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg_seg_Comp')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

In [ ]:
print("Model VGG 19 Comp Part D")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg_seg_Comp')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

### Part E

In [ ]:
X_P = np.load(P_dir + '/X_Covid_Cohen_C_Seg_I.npy')
X_train_P,X_test_P,X_dev_P = np.array_split(X_P,[int(0.7*len(X_P)),int(0.85*len(X_P))])

In [ ]:
X_N1 = np.load(N_dir+'/X_Covid_Normal_Seg_I.npy')
X_train_N1,X_test_N1,X_dev_N1 = np.array_split(X_N1,[int(0.7*len(X_N1)),int(0.85*len(X_N1))])
X_N2 = np.load(N_dir+'/X_Covid_Pneumo_Seg_I.npy')
X_train_N2,X_test_N2,X_dev_N2 = np.array_split(X_N2,[int(0.7*len(X_N2)),int(0.85*len(X_N2))])

In [ ]:
X_train_N=np.concatenate((X_train_N1,X_train_N2))
X_dev_N=np.concatenate((X_dev_N1,X_dev_N2))
X_test_N=np.concatenate((X_test_N1,X_test_N2))

In [ ]:
X_train=np.concatenate((X_train_N,X_train_P))
X_dev=np.concatenate((X_dev_N,X_dev_P))
X_test=np.concatenate((X_test_N,X_test_P))

In [ ]:
y_train_N=np.array([[1,0]]*len(X_train_N))
y_dev_N=np.array([[1,0]]*len(X_dev_N))
y_test_N=np.array([[1,0]]*len(X_test_N))

y_train_P=np.array([[0,1]]*len(X_train_P))
y_dev_P=np.array([[0,1]]*len(X_dev_P))
y_test_P=np.array([[0,1]]*len(X_test_P))

In [ ]:
y_train=np.concatenate((y_train_N,y_train_P))
y_dev=np.concatenate((y_dev_N,y_dev_P))
y_test=np.concatenate((y_test_N,y_test_P))

In [ ]:
#Min_max scaled
X_train,y_train=min_max_preprocessing(X_train,y_train)
X_dev,y_dev=min_max_preprocessing(X_dev,y_dev)
X_test,y_test=min_max_preprocessing(X_test,y_test)

In [ ]:
X_train = np.float32(X_train)
X_dev = np.float32(X_dev)
X_test = np.float32(X_test)

In [ ]:
#std normalization
X_train,y_train,mean,std=samplewise_preprocessing(X_train,y_train)

In [ ]:
X_dev=featurewise_preprocessing(X_dev,mean,std)
X_test=featurewise_preprocessing(X_test,mean,std)

#### Train Models (GPU) 

In [ ]:
log_dir="/content/drive/MyDrive/Covid19/COVID_FINAL/logs/"

In [ ]:
class_weight= get_weight(np.argmax(y_train,axis=-1))

In [ ]:
print("Getting Base Model")
model = get_model_VGG16_gray()

In [ ]:
model_name='VGG16_imagenet_Covid_pos_vs_neg_inv_Comp'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

In [ ]:
print("Getting Base Model")
model = get_model_VGG19_gray()

In [ ]:
model_name='VGG19_imagenet_Covid_pos_vs_neg_inv_Comp'
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger(log_dir + '/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev),class_weight=class_weight,callbacks=[csv_logger,checkpoint])

#### Prediction

In [ ]:
print("Model VGG 16 Comp Part E")
model = get_model_VGG16_gray()
model.load_weights(log_dir+'VGG16_imagenet_Covid_pos_vs_neg_inv_Comp')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)

In [ ]:
print("Model VGG 19 Comp Part E")
model = get_model_VGG19_gray()
model.load_weights(log_dir+'VGG19_imagenet_Covid_pos_vs_neg_inv_Comp')

In [ ]:
predictions=model.predict(X_test)
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

In [ ]:
metrics(y_hat,y_pred_bool)
labels = ["No-Covid","Covid"]
get_performance_metrics(y_test,predictions,labels, acc=get_accuracy, prevalence=get_prevalence, 
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

In [ ]:
#ROC_CURVE 
get_roc_curve(y_test,predictions,labels)